In [1]:
mountedDB = {}

## 欢迎进入 Notebook  

这里你可以编写代码，文档  

### 关于文件目录  


**project**：project 目录是本项目的工作空间，可以把将项目运行有关的所有文件放在这里，目录中文件的增、删、改操作都会被保留  


**input**：input 目录是数据集的挂载位置，所有挂载进项目的数据集都在这里，未挂载数据集时 input 目录被隐藏  


**temp**：temp 目录是临时磁盘空间，训练或分析过程中产生的不必要文件可以存放在这里，目录中的文件不会保存  


In [2]:
# 查看个人持久化工作区文件
!ls /home/mw/project/

details.parquet  rent.parquet  test.parquet  train.parquet  train_plus.parquet


In [3]:
# 查看当前挂载的数据集目录
!ls /home/mw/input/

__init__.py  quant4533


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

df = pd.read_parquet('/home/mw/project/train_plus.parquet')

# 数据预处理
# 转换需要作为分类变量的列
df['城市'] = df['城市'].astype('category')
df['区域'] = df['区域'].astype('category')
df['板块'] = df['板块'].astype('category')

# 定义特征和标签
X = df.drop(columns=['价格', '小区名称', '交易时间'])
y = df['价格']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=111)

# 定义预处理器
numeric_features = X.select_dtypes(include=['int32', 'int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['category']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', Pipeline([
        ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(degree=2, interaction_only=True))  # 二次项
    ]), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

In [5]:
from sklearn.model_selection import GridSearchCV

# 更新模型管道
models = {
    'Lasso': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Lasso(random_state=111))
    ]),
    'Ridge': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', Ridge(random_state=111))
    ]),
    'ElasticNet': Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', ElasticNet(random_state=111))
    ])
}

# 参数网格
param_grids = {
    'Lasso': {
        'regressor__alpha': [0.001, 0.01, 0.1, 1, 10],
        'regressor__max_iter': [5000, 10000],
        'regressor__tol': [1e-4, 1e-5]
    },
    'Ridge': {
        'regressor__alpha': [0.1, 1, 10, 100, 1000]
    },
    'ElasticNet': {
        'regressor__alpha': [0.001, 0.01, 0.1, 1],
        'regressor__l1_ratio': [0.2, 0.5, 0.8],
        'regressor__max_iter': [5000, 10000]
    }
}

# 执行网格搜索
best_models = {}
for name in models.keys():
    grid_search = GridSearchCV(models[name], param_grids[name], 
                              cv=5, scoring='neg_mean_squared_error',
                              n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"{name} best params: {grid_search.best_params_}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.563e+15, tolerance: 1.421e+12
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.631e+15, tolerance: 1.413e+12
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.560e+15, tolerance: 1.407e+12

Lasso best params: {'regressor__alpha': 1, 'regressor__max_iter': 10000, 'regressor__tol': 0.0001}
Fitting 5 folds for each of 5 candidates, totalling 25 fits


/opt/conda/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/conda/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/conda/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/conda/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/conda/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/conda/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/conda/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/opt/conda/lib/python3.9/site-packages/sklearn/utils/ex

Ridge best params: {'regressor__alpha': 0.1}
Fitting 5 folds for each of 24 candidates, totalling 120 fits


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.972e+15, tolerance: 1.421e+13
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.917e+15, tolerance: 1.407e+13
  model = cd_fast.enet_coordinate_descent(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.862e+15, tolerance: 1.405e+13

ElasticNet best params: {'regressor__alpha': 0.001, 'regressor__l1_ratio': 0.8, 'regressor__max_iter': 10000}
